In [ ]:
import time
import re
import time
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException
from extract.common import config
from extract.common import credentials
from extract.common import browser
from extract.log_in_page import LogInPage
from extract.project_list_page import CumploProjectListPage
from extract.project_list_page import BriqProjectListPage
from extract.project_page import CumploProjectPage
from extract.project_page import BriqProjectPage
from extract.factories import ProjectList
from extract.factories import Project

In [ ]:
site = 'cumplo'

login_page = LogInPage(site)
login_page.navigate()


In [ ]:
project_list_page = ProjectList().factory(site)
project_list_page.navigate()

payloads = []
project_page = Project().factory(site)

for url_project in project_list_page.project_urls:
    project_page.home = url_project
    project_page.navigate()
    payload = project_page.payload
    payloads.append(payload)
    print(payload)